In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy as scp
from dwave.samplers import SimulatedAnnealingSampler
import dimod
import docx
import hybrid

# Создаю бинарный вектор и ввожу слагаемые штрафов/бонусов

In [2]:
# Чтобы не сойти с ума при написании условий, вводится функция индексирования: по сути,
# искомый бинарный вектор |X> = |s m t d w g> [subject, professor, time, day, week, group].
# Задавать условия удобнее в шестимерном массиве

def ind(s, m, t, d, w, g):
    i = s + 2*m + 10*t + 80*d + 480*w + 960*g
    return i

In [4]:
#Матрица единичек, а то студенты вообще никуда не пойдут

Q = np.zeros((1920, 1920))
for i in range(1920):
    for j in range(1920):
        Q[i, j] = -1

#Ограничение от преподавателей (4 условие)

for g in range(2):
    for w in range(2):
        for t in range(8):
            for s in range(2):
                Q[ind(s, 0, t, 2, w, g), ind(s, 0, t, 2, w, g)] += 10000
                Q[ind(s, 1, t, 0, w, g), ind(s, 1, t, 0, w, g)] += 10000
                Q[ind(s, 2, t, 5, w, g), ind(s, 2, t, 5, w, g)] += 10000
                Q[ind(s, 3, t, 1, w, g), ind(s, 3, t, 1, w, g)] += 10000
                Q[ind(s, 4, t, 1, w, g), ind(s, 4, t, 1, w, g)] += 10000

#Ограничение на количество пар в день (2 условие)

for g in range(2):
    for w in range(2):
        for d in range(6):
            for m1 in range(5):
                for m2 in range(5):
                    for s1 in range(2):
                        for s2 in range(2):
                            for t in range(8):
                                for i in range(7):
                                    Q[ind(s1, m1, t, d, w, g), ind(s2, m2, (t + 1 + i) % 6, d, w, g)] += 300
                            
#Ограничение на количество пар по одному предмету в день (3 условие)

for g in range(2):
    for w in range(2):
        for d in range(6):
            for m in range(5):
                for s in range(2):
                    for t in range(8):
                        for i in range(7):
                            Q[ind(s, m, t, d, w, g), ind(s, m, (t + 1 + i) % 8, d, w, g)] += 5000
                            
#Буст, чтобы студенты вообще ходили на пары (условие на 4 пары)

for g in range(2):
    for w in range(2):
        for d1 in range(6):
            for m in range(5):
                for s in range(2):
                    for t1 in range(8):
                        for d2 in range(5):
                            for t2 in range(8):
                                Q[ind(s, m, t1, d1, w, g), ind(s, m, t2, (d1 + d2 + 1) % 6, w, g)] -= 1
                                
#Ограничение на ведение пары в двух группах одновременно (хоть условий таких и не было, вдруг там поточная :))

for w in range(2):
    for d in range(6):
        for m in range(5):
            for s1 in range(2):
                for t in range(8):
                    for g in range(2):
                        for s2 in range(2):
                            Q[ind(s1, m, t, d, w, g), ind(s2, m, t, d, w, (g + 1) % 2)] += 100
                            
#Ограничение на одновременные пары у группы

for g in range(2):
    for w in range(2):
        for d in range(6):
            for m1 in range(5):
                for m2 in range(5):
                    for s1 in range(2):
                        for s2 in range(2):
                            for t in range(8): 
                                Q[ind(s1, m1, t, d, w, g), ind(s2, m2, t, d, w, g)] += 1000
                                Q[ind(s1, m1, t, d, w, g), ind(s1, m1, t, d, w, g)] -= 1
                            
#Добавки для расплывчатости расписания

for g in range(2):
    for w in range(2):
        for d in range(6):
            for m1 in range(5):
                for s1 in range(2):
                    for t1 in range(8):
                        for t2 in range(8):
                            for m2 in range(5):
                                for s2 in range(2):
                                    Q[ind(s1, m1, t, d1, w, g), ind(s2, m2, t, d2, w, (g + 1) % 2)] -= 1

for g in range(2):
    for w in range(2):
        for d in range(6):
            for m1 in range(5):
                for s1 in range(2):
                    for t1 in range(8):
                        for t2 in range(8):
                            for m2 in range(5):
                                for s2 in range(2):
                                    Q[ind(s1, m1, t, d1, w, g), ind(s2, m2, t, d2, (w + 1) % 2, g)] -= 10
                                    
for g in range(2):
    for w in range(2):
        for d in range(6):
            for m1 in range(5):
                for s1 in range(2):
                    for t in range(8):
                        for m2 in range(4):
                            for s2 in range(2):
                                Q[ind(s1, m1, t, d, w, g), ind(s2, (m1 + m2 + 1) % 5, t, d, w, g)] -= 100                    
                        

print(Q)

[[ 1.2890e+03  1.2990e+03  1.1990e+03 ... -1.0000e+00 -1.0000e+00
  -1.0000e+00]
 [ 1.2990e+03  1.2890e+03  1.1990e+03 ... -1.0000e+00 -1.0000e+00
  -1.0000e+00]
 [ 1.1990e+03  1.1990e+03  1.1289e+04 ... -1.0000e+00 -1.0000e+00
  -1.0000e+00]
 ...
 [-1.0000e+00 -1.0000e+00 -1.0000e+00 ...  9.8900e+02  8.9900e+02
   8.9900e+02]
 [-1.0000e+00 -1.0000e+00 -1.0000e+00 ...  8.9900e+02  9.8900e+02
   9.9900e+02]
 [-1.0000e+00 -1.0000e+00 -1.0000e+00 ...  8.9900e+02  9.9900e+02
   9.8900e+02]]


# Минимизация вектора через библиотеку от dwave

In [5]:
sampler = SimulatedAnnealingSampler()
qubo = Q
sampleset = sampler.sample_qubo(qubo, num_reads = 10)

In [6]:
#сборка бинарного вектора в шестимерный массив, для того же удобства теперь уже при создании таблицы

a = sampleset.first.sample
Table = np.zeros((((((2, 5, 6, 8, 2, 2))))))
for i in range(1920):
    Table[i // 960, (i % 960) // 192, (i % 192) // 32, (i % 32) // 4, (i % 4) // 2, i % 2] = a[i]

In [7]:
#Окно проверки на адекватность

(n1, n2, n3, n4) = (0, 0, 0, 0)

for g in range(2):
    for w in range(2):
        for t in range(8):
            for d in range(6):
                for m in range(5):
                    for s in range(2):
                        if Table[s, m, d, t, w, g] == 1:
                            if g == 0:
                                n1 += 1
                            else:
                                n2 += 1
                            if w == 0:
                                n3 += 1
                            else:
                                n4 += 1
                                
print(n1, n2, n3, n4)

40 40 32 48


# Собираю таблицу данных на основе минимизированного вектора

In [9]:
#Собираем таблицу

doc = docx.Document()
table = doc.add_table(rows = n1 + n2 + 1, cols = 7)
table.stile = 'Table Grid'

day = ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday')
time = ('09:00 - 10:00', '10:00 - 11:00', '11:00 - 12:00', '12:00 - 13:00', '13:00 - 14:00', '14:00 - 15:00', '15:00 - 16:00', '16:00 - 17:00')
name = ('Иванов', 'Петров', 'Сидоров', 'Карпов', 'Соколов')
subject = ('Квантовая механика', 'Квантовая теория информации', 'Квантовые вычисления', 'Сложность квантовых алгоритмов', 
           'Квантовые алгоритмы в логистике', 'Квантовое машинное обучение', 'Моделирование квантовых систем', 
           'Квантовые алгоритмы в химии', 'Физическая реализация квантовых компьютеров', 'Моделирование квантовых алгоритмов')

(col, row) = (1, 1)
table.cell(0, 1).text = str('Group')
table.cell(0, 2).text = str('Week')
table.cell(0, 3).text = str('Day')
table.cell(0, 4).text = str('Hours')
table.cell(0, 5).text = str('Subject')
table.cell(0, 6).text = str('Name')

for i in range(n1 + n2 + 1):
    table.cell(i, 0).text = str(i)
    
for g in range(2):
    for w in range(2):
        for d in range(6):
            for t in range(8):
                for m in range(5):
                    for s in range(2):
                        if Table[s, m, d, t, w, g] == 1:
                            table.cell(row, 1).text = str('QC' + str(g + 1))
                            table.cell(row, 2).text = str('Week' + str(w + 1))
                            table.cell(row, 3).text = str(day[d])
                            table.cell(row, 4).text = str(time[t])
                            table.cell(row, 5).text = str(subject[2*m+s])
                            table.cell(row, 6).text = str(name[m])
                            row += 1


doc.save('C:\\Users\\pasht\\Desktop\\Table.docx')